In [3]:
import pandas as pd
import numpy as np

csv_name_altersklassen = "2019_altersklassen.csv"

df = pd.read_csv(csv_name_altersklassen, encoding='ISO-8859-1')

print(df.columns)

df = df.drop(columns=["Jahr", "Bevölkerungstyp", "Geschlecht", "Zivilstand"])

df = df.rename(columns={'Kanton (-) / Bezirk (>>) / Gemeinde (......)': 'Gemeinde'})

df['Kanton'] = np.nan
df['Bezirk'] = np.nan

for index, row in df.iterrows():
    if row['Gemeinde'].startswith('-'):
        current_kanton = row['Gemeinde'][1:].strip()
    elif row['Gemeinde'].startswith('>>'):
        current_bezirk = row['Gemeinde'][2:].strip()
    elif row['Gemeinde'].startswith('......'):
        
        df.at[index, 'Kanton'] = current_kanton
        df.at[index, 'Bezirk'] = current_bezirk
        
gemeinde_df = df[df['Gemeinde'].str.startswith('......')].copy()
gemeinde_df['Gemeinde'] = gemeinde_df['Gemeinde'].map(lambda x: x[6:].strip())

gemeinde_df[['Gemeindenummer', 'Gemeinde']] = gemeinde_df['Gemeinde'].str.split(' ', n=1, expand=True)

gemeinde_df = gemeinde_df.drop(columns=["Gemeindenummer"])

gemeinde_df.to_csv("2019_altersklassen_absolute.csv", index=False)

non_numeric_columns = ["Gemeinde", "Kanton", "Bezirk"]


numeric_gemeinde_df = gemeinde_df.drop(columns=non_numeric_columns)

gemeinde_df['Total Population'] = numeric_gemeinde_df.sum(axis=1)
for column in numeric_gemeinde_df.columns:
    gemeinde_df[column] = gemeinde_df[column] / gemeinde_df['Total Population']

gemeinde_df = gemeinde_df.drop(columns=["Total Population"])




df_municipalities = pd.read_csv("regions_municipalities.csv", sep=";")
# Match the GEO_NAME of df_municipalities to Gemeinde of df and add a column called Sprachregion to df based on DESC_VAL of df_municipalities

for i, row in gemeinde_df.iterrows():
        df_filtered = df_municipalities[df_municipalities["GEO_NAME"] == row["Gemeinde"]]
        if len(df_filtered) > 0:
            gemeinde_df.at[i, "Sprachregion"] = df_filtered["DESC_VAL"].values[0]
        else:
            gemeinde_df.at[i, "Sprachregion"] = "Unknown"
            print(row["Gemeinde"])

gemeinde_df.to_csv("2019_altersklassen_relative.csv", index=False)


Index(['Jahr', 'Kanton (-) / Bezirk (>>) / Gemeinde (......)',
       'Bevölkerungstyp', 'Geschlecht', 'Zivilstand', '0-4 Jahre', '5-9 Jahre',
       '10-14 Jahre', '15-19 Jahre', '20-24 Jahre', '25-29 Jahre',
       '30-34 Jahre', '35-39 Jahre', '40-44 Jahre', '45-49 Jahre',
       '50-54 Jahre', '55-59 Jahre', '60-64 Jahre', '65-69 Jahre',
       '70-74 Jahre', '75-79 Jahre', '80-84 Jahre', '85-89 Jahre',
       '90-94 Jahre', '95-99 Jahre', '100 Jahre und mehr'],
      dtype='object')
Bois-d'Amont
Obersaxen Mundaun
